In [9]:
import zipfile

with zipfile.ZipFile('BC NEW dataset (400X).zip','r') as zip_ref:
    zip_ref.extractall('/content/')

In [39]:
import os

def count_images_in_folder(folder_path):
    """Counts the number of images in a given folder.

    Args:
        folder_path (str): The path to the folder.

    Returns:
        int: The number of images in the folder.
    """

    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed
    image_count = 0

    for filename in os.listdir(folder_path):
        if any(filename.endswith(ext) for ext in image_extensions):
            image_count += 1

    return image_count

# Replace 'your_folder_path' with the actual path to your folder
folder_path = "C:/benignlow400x"
#folder_path ="C:/benignhigh400x"
#folder_path = "C:/benignmedium400x"
#folder_path = "C:/benign400Mx"
#folder_path = "C:/content/BC NEW dataset (400X)/maligant"
total_images = count_images_in_folder(folder_path)
print("Total images in the folder:", total_images)

Total images in the folder: 2154


In [6]:
#400x calculation of density
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

def calculate_density(image, microns_per_pixel_x, microns_per_pixel_y, apply_blur=False):
    """
    Calculate nuclei density in nuclei per square millimeter using traditional image processing.

    Parameters:
    - image (numpy.ndarray): Input histopathology image.
    - microns_per_pixel_x (float): Microns per pixel along X-axis.
    - microns_per_pixel_y (float): Microns per pixel along Y-axis.
    - apply_blur (bool): Apply Gaussian blur to the image to reduce noise (default is False).

    Returns:
    - num_nuclei (int): Number of nuclei detected.
    - density_per_mm2 (float): Nuclei density per square millimeter.
    - binary_mask (numpy.ndarray): Binary mask of detected nuclei.
    """
    # Convert to grayscale if the image is not already in grayscale
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    # Optionally apply Gaussian blur to reduce noise
    if apply_blur:
        gray_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Apply Otsu's thresholding to binarize the image
    _, binary_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find connected components (i.e., nuclei)
    num_labels, labels_im = cv2.connectedComponents(binary_mask)

    # Number of nuclei is num_labels - 1 because one label is for the background
    num_nuclei = num_labels - 1

    # Calculate the image area in square microns
    height, width = binary_mask.shape
    area_sq_microns = height * microns_per_pixel_y * width * microns_per_pixel_x

    # Convert area to square millimeters
    area_sq_mm = area_sq_microns / 1e6

    # Calculate nuclei density (nuclei per square millimeter)
    density_per_mm2 = num_nuclei / area_sq_mm

    return num_nuclei, density_per_mm2, binary_mask

def split_image_into_patches(image, patch_size):
    """
    Split the image into overlapping patches.

    Parameters:
    - image (numpy.ndarray): Input image.
    - patch_size (int): Size of each square patch.

    Returns:
    - patches (list): List of image patches.
    """
    patches = []
    height, width = image.shape[:2]

    # Create non-overlapping patches
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            patch = image[y:y + patch_size, x:x + patch_size]
            # Check if patch is of the desired size
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)
    return patches

def process_images_in_folder(folder_path, microns_per_pixel_x, microns_per_pixel_y, patch_size):
    """
    Process all images in a specified folder to calculate nuclei density.

    Parameters:
    - folder_path (str): Path to the folder containing images.
    - microns_per_pixel_x (float): Microns per pixel along X-axis.
    - microns_per_pixel_y (float): Microns per pixel along Y-axis.
    - patch_size (int): Size of the patches to split images into.
    """
    # Get all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Create an empty list to store the overall results
    all_density_results = []

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        # Load the histopathology image
        image = cv2.imread(image_path)

        # Split image into patches
        patches = split_image_into_patches(image, patch_size)

        # Process each patch and calculate nuclei density
        for idx, patch in enumerate(patches):
            num_nuclei, density_mm2, binary_mask = calculate_density(patch, microns_per_pixel_x, microns_per_pixel_y, apply_blur=True)

            # Append results for the current patch
            all_density_results.append({
                'image_file': image_file,
                'patch_index': idx,
                'num_nuclei': num_nuclei,
                'density_per_mm2': density_mm2
            })

            # Define a folder based on density ranges
            if density_mm2 < 70000 and density_mm2 > 1:
                density_folder = "Low Density"
            elif density_mm2 < 10000 and density_mm2 > 70000:
                density_folder = "Medium Density"
            else:
                density_folder = "High Density"

            # Create a directory for the density range if it doesn't exist
            density_path = os.path.join("nuclei_density_results200ffxfinal(29.10.2024)", density_folder)
            os.makedirs(density_path, exist_ok=True)

            # Save the patch in the corresponding folder
            patch_filename = os.path.join(density_path, f"{image_file}_patch_{idx + 1}.png")
            cv2.imwrite(patch_filename, patch)

            # Print the nuclei count and density for each patch
            print(f"Image: {image_file}, Patch {idx + 1}:")
            print(f"Number of nuclei: {num_nuclei}")
            print(f"Nuclei density: {density_mm2:.2f} nuclei per mm²")
            print("-" * 30)

    # Convert results to a DataFrame
    df_density = pd.DataFrame(all_density_results)

    # Create output directory if it doesn't exist
    output_dir = "nuclei_density_results200ffx(final29.10.2024)"
    os.makedirs(output_dir, exist_ok=True)

    # Save all results to a single CSV file
    output_csv = os.path.join(output_dir, "combined_density_results200ffx(final29.10.2024).csv")
    df_density.to_csv(output_csv, index=False)

    # Print the first few rows of the dataframe
    print("Combined density results saved to:", output_csv)
    print(df_density.head())

# Example usage
folder_path = "E:/BC NEW dataset (200X)/maligant"  # Replace with your folder path

# Set microns per pixel (example values, replace with actual values from metadata if available)
MICRONS_PER_PIXEL_X = 0.0325 # Example value, replace with actual value from image metadata
MICRONS_PER_PIXEL_Y = 0.0325  # Example value, replace with actual value from image metadata
PATCH_SIZE = 224  # Unique patch size (224x224 pixels)

# Process all images in the folder
process_images_in_folder(folder_path, MICRONS_PER_PIXEL_X, MICRONS_PER_PIXEL_Y, PATCH_SIZE)



Image: SOB_M_DC-14-11031-200-001.png, Patch 1:
Number of nuclei: 38
Nuclei density: 717002.78 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-11031-200-001.png, Patch 2:
Number of nuclei: 58
Nuclei density: 1094372.66 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-11031-200-001.png, Patch 3:
Number of nuclei: 32
Nuclei density: 603791.81 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-11031-200-001.png, Patch 4:
Number of nuclei: 44
Nuclei density: 830213.74 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-11031-200-001.png, Patch 5:
Number of nuclei: 60
Nuclei density: 1132109.65 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-11031-200-001.png, Patch 6:
Number of nuclei: 69
Nuclei density: 1301926.10 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-11031-200-002.png, Patch 1:
Number of nuclei: 41
Nuclei density: 773608.26 nuclei per mm²
------------------------------
Image: SOB_M_DC-1

In [21]:
import os

def count_images(folder_path):
  """Counts the number of image files in a given folder.

  Args:
    folder_path: The path to the folder containing the images.

  Returns:
    The number of image files in the folder.
  """

  image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add other image extensions if needed
  image_count = 0

  for file_name in os.listdir(folder_path):
    file_extension = os.path.splitext(file_name)[1]
    if file_extension.lower() in image_extensions:
      image_count += 1

  return image_count

# Replace 'folder1', 'folder2', and 'folder3' with the actual paths to your folders
folder1_path = 'nuclei_density_results400ffx/Low Density'
folder2_path = 'nuclei_density_results400ffx/Medium Density'
folder3_path = 'nuclei_density_results400ffx/High Density'

image_count_folder1 = count_images(folder1_path)
image_count_folder2 = count_images(folder2_path)
image_count_folder3 = count_images(folder3_path)

print("Number of images in folder 1:", image_count_folder1)
print("Number of images in folder 2:", image_count_folder2)
print("Number of images in folder 3:", image_count_folder3)

Number of images in folder 1: 2136
Number of images in folder 2: 1158
Number of images in folder 3: 2856
